In [ ]:
import cv2
import torch
from PIL import Image
from t_diffusers.unet_2d_condition import UNet2DConditionModel

from diffusers import AutoencoderKL, DDPMScheduler

from .pipeline_text_diffuser_sd15 import StableDiffusionPipeline


td_ckpt = "GoGiants1/td-unet15"

vae = AutoencoderKL.from_pretrained(
    "runwayml/stable-diffusion-v1-5"
)
unet = UNet2DConditionModel(
    td_ckpt,
    subfolder="unet",
)

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    vae=vae,
    unet=unet,
    torch_dtype=torch.float32,
)
pipe.load_ip_adapter(
    "h94/IP-Adapter",
    subfolder=[
        "models",
    ],
    weight_name=[
        "ip-adapter_sd15.safetensors",
    ],
)
pipe.set_ip_adapter_scale(0.5)

text_mask_image = cv2.imread("text_mask.png")

input_image = Image.open("original_image.png").convert("RGB").resize((512, 512))

pipe.scheduler = DDPMScheduler.from_config(pipe.scheduler.config)
generator = torch.Generator(device="cuda").manual_seed(42)
pipe(
    prompt="A painting of a cat with a text mask",
    input_image=input_image,
    text_mask_image=text_mask_image,
    ip_adapter_image = input_image,
    width=512,
    height=512,
    guidance_scale=7,
    generator=generator,
)


: 